# G-Score Strategy - Investments Committee Quant Sector


In [1]:
from polygon import RESTClient
from collections import defaultdict
import csv
import statistics
import datetime
from dateutil.relativedelta import relativedelta
import ipynb.fs.defs.sec as sec

#not easily able to access shared files so resorting to link download instead
#from google.colab import drive
#drive.mount("/content/drive", force_remount=True)

client = RESTClient("uwQtl3txGt5BLbecq7ZbIu0ZbuitCGjc")

In [2]:
def get_g_scores(query_date: str):
  tickers = {}
  with open('s&p500.csv', mode ='r') as file:
    tickers = {k:v for (k,v) in csv.reader(file)} # convert csv to dict

  ticker_data = defaultdict(lambda: {})
  industry_data = defaultdict(lambda: defaultdict(lambda: []*13))

  for t in tickers:
    try:
      ticker_data[t]["Price"] = client.get_aggs(ticker=t, multiplier=1, timespan='day', from_=query_date, to=query_date)[0].close
      ticker_data[t]["Volume"] = client.get_aggs(ticker=t, multiplier=1, timespan='day', from_=query_date, to=query_date)[0].volume
      ticker_data[t]["P/E"] = ticker_data[t]["Price"] / (sec.get_net_income(t, query_date) / sec.get_shares_outstanding(t, query_date))
      ticker_data[t]["EV"] = sec.get_shares_outstanding(t, query_date) * ticker_data[t]["Price"]+ sec.get_current_liabilities(t, query_date) - sec.get_current_assets(t, query_date)
      ticker_data[t]["EBITDA"] = sec.get_ebit(t, query_date) + sec.get_depreciation_and_amortization(t, query_date)
      ticker_data[t]["EV/EBITDA"] = ticker_data[t]["EV"] / ticker_data[t]["EBITDA"]
      ticker_data[t]["P/B"] = ticker_data[t]["Price"] / ((sec.get_current_assets(t, query_date) - sec.get_current_liabilities(t, query_date)) / sec.get_shares_outstanding(t, query_date))
      ticker_data[t]["ROA"] = sec.get_net_income(t, query_date) / sec.get_current_assets(t, query_date)
      ticker_data[t]["CFROA"] = sec.get_ufcf(t, query_date) / sec.get_current_assets(t, query_date) # need net cash flow
      # R&D?
      ticker_data[t]["SMA"] = client.get_sma(ticker=t, timestamp = query_date).values[0].value
      ticker_data[t]["EMA"] = client.get_ema(ticker=t, timestamp = query_date).values[0].value
      ticker_data[t]["MACD"] = client.get_macd(ticker=t, timestamp = query_date).values[0].value
      ticker_data[t]["RSI"] = client.get_rsi(ticker=t, timestamp = query_date).values[0].value
      for k, v in ticker_data[t].items(): # add values to industry-specific data
        industry_data[tickers[t]][k].append(v)
    except:
      ticker_data.pop(t, None)

  print(query_date + ": " + str(len(ticker_data.keys())))

  for ind, values in industry_data.items():
    for category, data in values.items():
      industry_data[ind][category] = statistics.median(data)
  
  weightings = {"P/E": .15, "EV/EBITDA": .1, "P/B": .1, "ROA": .175, "CFROA": .175, "MACD": .15, "RSI": .15}
  comparisonTypes = {"P/E": '<', "EV/EBITDA": '<', "P/B": '<', "ROA": '>', "CFROA": '>', "MACD": '>', "RSI": '<'}
  # Dictates how ticker_val and industry_val are compared: ticker_val[P/E] < or > industry_val[P/E]

  def compare(t, category): 
    (ticker_val, industry_val) = (ticker_data[t][category], industry_data[tickers[t]][category])
    return ticker_val > industry_val if comparisonTypes[category] == '>' else ticker_val < industry_val

  g_scores = {t : sum(weight for (category, weight) in weightings.items() if compare(t, category)) 
              for t in ticker_data.keys()}
  return g_scores

In [18]:
def backtest(g_scores_list, timeframe, starting_date):
  curr_date = starting_date
  percent_change = 0
  curr_holdings = {}
  for g_scores in g_scores_list:
    curr_change = 0
    if curr_date != starting_date:
      for s in curr_holdings.keys():
        curr_price = client.get_aggs(ticker=s, multiplier=1, timespan='day', from_=curr_date.strftime('%Y-%m-%d'), to=curr_date.strftime('%Y-%m-%d'))[0].close
        curr_change += (curr_price - curr_holdings[s]) / curr_holdings[s]
      curr_change /= len(curr_holdings.keys())
    percent_change += curr_change
    curr_holdings.clear()

    num_stocks = 0
    prev_score = 0
    for stock in reversed(sorted(g_scores.items(), key=lambda x:x[1])):
      if stock[1] < prev_score and num_stocks >= 5:
        break
      curr_holdings[stock[0]] = client.get_aggs(ticker=stock[0], multiplier=1, timespan='day', from_=curr_date.strftime('%Y-%m-%d'), to=curr_date.strftime('%Y-%m-%d'))[0].close
      prev_score = stock[1]
      num_stocks += 1

    print("Portfolio Update: " + curr_date.strftime("%Y-%m-%d"))
    for s in curr_holdings.keys():
      print(s + ": " + str(curr_holdings[s]))
    if timeframe == "annual":
      curr_date += relativedelta(months=+12)
    elif timeframe == "quarterly":
      curr_date += relativedelta(months=+3)
  
  print("Overall P/L: " + str(percent_change * 100))
  curr_date += relativedelta(months=-3)
  spy_start = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=starting_date.strftime('%Y-%m-%d'), to=starting_date.strftime('%Y-%m-%d'))[0].close
  spy_end = client.get_aggs(ticker="SPY", multiplier=1, timespan='day', from_=curr_date.strftime('%Y-%m-%d'), to=curr_date.strftime('%Y-%m-%d'))[0].close
  print("SPY P/L: " + str((spy_end - spy_start) / spy_start * 100))

The following code runs the backtest for the last 5 years

In [4]:
query_date = datetime.datetime.strptime("2022-12-01", '%Y-%m-%d')
end_date = datetime.datetime.strptime("2023-03-01", '%Y-%m-%d')
g_scores_list = []
while query_date <= end_date:
    g_scores_list.append(get_g_scores(query_date.strftime("%Y-%m-%d")))
    query_date += relativedelta(months=+3)
print(g_scores_list)

2022-12-01: 301
2023-03-01: 306
[{'MMM': 0.85, 'ABT': 0.15, 'ABBV': 0.42500000000000004, 'ATVI': 0.32499999999999996, 'ADM': 0.4, 'ADBE': 0.5, 'AAP': 0.4, 'AES': 0.15, 'A': 0.5, 'APD': 0.5, 'AKAM': 0.7, 'ALK': 0.5, 'ALB': 0.15, 'ALLE': 0.75, 'LNT': 0.75, 'MO': 0.45, 'AMZN': 0.6749999999999999, 'AMCR': 0, 'AMD': 0.475, 'AEE': 0.5750000000000001, 'AAL': 0.5, 'AMT': 0.425, 'ABC': 0.5, 'AME': 0.5, 'AMGN': 0.6, 'APH': 0.1, 'ANSS': 0.32499999999999996, 'AON': 0.4, 'AAPL': 0.75, 'AMAT': 0.6749999999999999, 'APTV': 0.4, 'T': 0.7, 'ATO': 0.4, 'AZO': 0.6, 'AVY': 0.7999999999999999, 'BBWI': 0.6749999999999999, 'BAX': 0.5, 'BDX': 0.32499999999999996, 'BBY': 0.85, 'TECH': 0.85, 'BIIB': 0.4, 'BA': 0.65, 'BKNG': 0.15, 'BWA': 0.35, 'BSX': 0.175, 'BMY': 0.425, 'AVGO': 0.32499999999999996, 'BR': 0.5, 'BRO': 0.25, 'CDNS': 0.5, 'CZR': 0.425, 'CPB': 0.75, 'CAH': 0.4, 'KMX': 0.5, 'CCL': 0.5, 'CARR': 0.35, 'CBOE': 0.85, 'CE': 1.0, 'CNC': 0.25, 'CNP': 0.25, 'CF': 0.75, 'CRL': 0.75, 'CVX': 0.75, 'CHD': 0.85, '

In [19]:
backtest(g_scores_list, "quarterly", datetime.datetime.strptime("2022-12-01", '%Y-%m-%d'))

Portfolio Update: 2022-12-01
UNP: 215.93
TER: 92.29
TEL: 126.8
QRVO: 97.25
LRCX: 463.14
LH: 241.77
DRI: 145.65
CE: 107.49
Portfolio Update: 2023-03-01
TGT: 162.4
KHC: 38.4
CCI: 128.96
STX: 64.12
PFE: 40.18
MAS: 51.61
HCA: 244.3
Overall P/L: 3.0824531439168945
SPY P/L: -3.102754185281552
